In [1]:
import numpy as np
import lightgbm as lgb
from sklearn.datasets import load_boston
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import metrics,ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error as MSLE
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import random
import csv
import pickle
from sklearn.preprocessing import StandardScaler
random.seed(123)
#pd.set_option("display.max_columns", None)
from sklearn.preprocessing import LabelEncoder
from sklearn import svm 
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score

import os, time, glob, socket,random,csv,pickle
import pandas as pd

from timeit import Timer
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.decomposition import PCA 

pd.set_option("display.max_columns", None)
os.getcwd()
os.chdir(r'D:/fintech/')
path=os.getcwd()
import re

In [2]:
# A

y_train=pd.read_csv(path+'/grade/B/y_trainB.csv',index_col=0)
y_test=pd.read_csv(path+'/grade/B/y_testB.csv',index_col=0)
X_test=pd.read_csv(path+'/grade/B/X_testB.csv',index_col=0)
X_train=pd.read_csv(path+'/grade/B/X_trainB.csv',index_col=0)

In [3]:
print(y_train.shape)
print(y_test.shape)

(619370, 2)
(155106, 2)


In [145]:
print(X_train.shape)
print(X_test.shape)

(619370, 80)
(155106, 80)


imp=['Unnamed: 0.1','7','8','debt_settlement_flag_Y','20','0','12','9','11','hardship_flag_unknown']

X_test=X_test[imp]
X_train=X_train[imp]


In [141]:
X_trainall=pd.read_csv('X_train.csv')
X_testall=pd.read_csv('X_test.csv')

In [146]:
X_trainall['grade']=X_trainall['grade'].fillna(X_trainall['grade'].mode()[0])
X_testall['grade']=X_testall['grade'].fillna(X_testall['grade'].mode()[0])


In [147]:
print(X_trainall.shape)
print(X_testall.shape)

(2132800, 150)
(533200, 150)


In [148]:
filter=X_trainall.grade=='B'
X_trainall_B=X_trainall[filter]


filter=X_testall.grade=='B'
X_testall_B=X_testall[filter]

In [149]:
print(X_trainall_B.shape)
print(X_testall_B.shape)

(619370, 150)
(155106, 150)


In [154]:
X_trainall_B=X_trainall_B.rename(columns={'Unnamed: 0':'Unnamed: 0.1'})
X_testall_B=X_testall_B.rename(columns={'Unnamed: 0':'Unnamed: 0.1'})

In [155]:
#earliest_cr_line=借款人最早報告的信用額度被打開的月份
#issue_d=貸款資金的月份
#last_credit_pull_d=最近一個月 LC 為這筆貸款拉了貸
#last_pymnt_d=上個月收到付

p=['Unnamed: 0.1','emp_title','earliest_cr_line','issue_d','last_credit_pull_d','last_pymnt_d','addr_state']
X_train_add=X_trainall_B[p]


p=['Unnamed: 0.1','emp_title','earliest_cr_line','issue_d','last_credit_pull_d','last_pymnt_d','addr_state']
X_test_add=X_testall_B[p]


In [156]:
from datetime import datetime
import _strptime

In [157]:
all_d=X_train_add

all_d['last_credit_pull_d_trans'] = pd.to_datetime(all_d['last_credit_pull_d'], format= "%b-%y")
all_d['last_credit_pull_d_year'] = pd.DatetimeIndex(all_d['last_credit_pull_d_trans']).year
all_d['last_credit_pull_d_month'] =pd.DatetimeIndex(all_d['last_credit_pull_d_trans']).month
all_d.drop(columns=['last_credit_pull_d'])

all_d['last_pymnt_d_trans'] = pd.to_datetime(all_d['last_pymnt_d'], format= "%b-%y")
all_d['last_pymnt_d_year'] = pd.DatetimeIndex(all_d['last_pymnt_d_trans']).year
all_d['last_pymnt_d_month'] =pd.DatetimeIndex(all_d['last_pymnt_d_trans']).month
all_d.drop(columns=['last_pymnt_d'])

all_d['issue_d_trans'] = pd.to_datetime(all_d['issue_d'], format= "%b-%y")
all_d['issue_d_year'] = pd.DatetimeIndex(all_d['issue_d_trans']).year
all_d['issue_d_month'] =pd.DatetimeIndex(all_d['issue_d_trans']).month
all_d.drop(columns=['issue_d'])

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,Unnamed: 0.1,emp_title,earliest_cr_line,last_credit_pull_d,last_pymnt_d,addr_state,last_credit_pull_d_trans,last_credit_pull_d_year,last_credit_pull_d_month,last_pymnt_d_trans,last_pymnt_d_year,last_pymnt_d_month,issue_d_trans,issue_d_year,issue_d_month
2,1192524,GENERAL WAREHOUSE,Jun-06,Aug-20,Aug-20,IN,2020-08-01,2020.0,8.0,2020-08-01,2020.0,8.0,2019-08-01,2019.0,8.0
9,2062903,Financial Aid Tech,Jan-94,Aug-20,Jan-20,CA,2020-08-01,2020.0,8.0,2020-01-01,2020.0,1.0,2015-12-01,2015.0,12.0
13,653911,Financial Systems Analyst,Sep-09,Feb-20,Jul-19,MA,2020-02-01,2020.0,2.0,2019-07-01,2019.0,7.0,2017-07-01,2017.0,7.0
14,1985095,Quality Control Director,Oct-05,Mar-20,Feb-20,CA,2020-03-01,2020.0,3.0,2020-02-01,2020.0,2.0,2016-09-01,2016.0,9.0
19,1223705,CAP TEAM SUPERVISOR,Jan-92,Aug-20,Sep-18,TN,2020-08-01,2020.0,8.0,2018-09-01,2018.0,9.0,2016-06-01,2016.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2132787,1770366,Waitress,Oct-06,Jun-20,Feb-20,CA,2020-06-01,2020.0,6.0,2020-02-01,2020.0,2.0,2019-04-01,2019.0,4.0
2132789,1195738,Sr. Construction Manager,Jun-06,Mar-20,Oct-18,OR,2020-03-01,2020.0,3.0,2018-10-01,2018.0,10.0,2016-01-01,2016.0,1.0
2132790,178207,Owner,Feb-92,Aug-20,Apr-17,TX,2020-08-01,2020.0,8.0,2017-04-01,2017.0,4.0,2014-04-01,2014.0,4.0
2132792,935,Regional Training Coordinator,Dec-04,Mar-20,Feb-20,SC,2020-03-01,2020.0,3.0,2020-02-01,2020.0,2.0,2017-04-01,2017.0,4.0


In [158]:
all_d=X_test_add

all_d['last_credit_pull_d_trans'] = pd.to_datetime(all_d['last_credit_pull_d'], format= "%b-%y")
all_d['last_credit_pull_d_year'] = pd.DatetimeIndex(all_d['last_credit_pull_d_trans']).year
all_d['last_credit_pull_d_month'] =pd.DatetimeIndex(all_d['last_credit_pull_d_trans']).month
all_d.drop(columns=['last_credit_pull_d'])

all_d['last_pymnt_d_trans'] = pd.to_datetime(all_d['last_pymnt_d'], format= "%b-%y")
all_d['last_pymnt_d_year'] = pd.DatetimeIndex(all_d['last_pymnt_d_trans']).year
all_d['last_pymnt_d_month'] =pd.DatetimeIndex(all_d['last_pymnt_d_trans']).month
all_d.drop(columns=['last_pymnt_d'])

all_d['issue_d_trans'] = pd.to_datetime(all_d['issue_d'], format= "%b-%y")
all_d['issue_d_year'] = pd.DatetimeIndex(all_d['issue_d_trans']).year
all_d['issue_d_month'] =pd.DatetimeIndex(all_d['issue_d_trans']).month
all_d.drop(columns=['issue_d'])

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,Unnamed: 0.1,emp_title,earliest_cr_line,last_credit_pull_d,last_pymnt_d,addr_state,last_credit_pull_d_trans,last_credit_pull_d_year,last_credit_pull_d_month,last_pymnt_d_trans,last_pymnt_d_year,last_pymnt_d_month,issue_d_trans,issue_d_year,issue_d_month
0,1272824,Optical scanner,Jul-79,Aug-20,Aug-20,CA,2020-08-01,2020.0,8.0,2020-08-01,2020.0,8.0,2019-11-01,2019.0,11.0
6,1827794,Admin Assistant,May-03,Aug-20,Sep-19,CA,2020-08-01,2020.0,8.0,2019-09-01,2019.0,9.0,2017-05-01,2017.0,5.0
8,2484284,Materials Planner,Jul-88,Aug-20,Aug-20,MN,2020-08-01,2020.0,8.0,2020-08-01,2020.0,8.0,2019-08-01,2019.0,8.0
13,574678,Opperation manager,Sep-95,Aug-20,Jun-18,IL,2020-08-01,2020.0,8.0,2018-06-01,2018.0,6.0,2016-03-01,2016.0,3.0
18,2081350,Deputy,Aug-10,Dec-18,Dec-18,VA,2018-12-01,2018.0,12.0,2018-12-01,2018.0,12.0,2016-11-01,2016.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533176,364310,Grounds Foreman,May-91,Jan-19,Jan-19,NY,2019-01-01,2019.0,1.0,2019-01-01,2019.0,1.0,2016-12-01,2016.0,12.0
533184,659648,Librarian II,Mar-80,May-19,Nov-18,MO,2019-05-01,2019.0,5.0,2018-11-01,2018.0,11.0,2017-09-01,2017.0,9.0
533188,659291,Photographer,Aug-01,Aug-20,Aug-20,IL,2020-08-01,2020.0,8.0,2020-08-01,2020.0,8.0,2015-11-01,2015.0,11.0
533196,169028,NaN,May-06,Aug-20,Jun-19,MI,2020-08-01,2020.0,8.0,2019-06-01,2019.0,6.0,2019-01-01,2019.0,1.0


In [159]:
all_d=X_train_add

all_d['issue_To_last_pymnt']=all_d['issue_d_trans']-all_d['last_pymnt_d_trans']
all_d['last_credit_pull_d_To_last_pymnt']=all_d['last_credit_pull_d_trans']-all_d['last_pymnt_d_trans']
all_d['issue_To_last_credit_pull_d']=all_d['issue_d_trans']-all_d['last_credit_pull_d_trans']

all_d['issue_To_last_pymnt']=all_d['issue_To_last_pymnt'].map(lambda x:x.days)
all_d['last_credit_pull_d_To_last_pymnt']=all_d['last_credit_pull_d_To_last_pymnt'].map(lambda x:x.days)
all_d['issue_To_last_credit_pull_d']=all_d['issue_To_last_credit_pull_d'].map(lambda x:x.days)


C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [160]:
all_d=X_test_add

all_d['issue_To_last_pymnt']=all_d['issue_d_trans']-all_d['last_pymnt_d_trans']
all_d['last_credit_pull_d_To_last_pymnt']=all_d['last_credit_pull_d_trans']-all_d['last_pymnt_d_trans']
all_d['issue_To_last_credit_pull_d']=all_d['issue_d_trans']-all_d['last_credit_pull_d_trans']

all_d['issue_To_last_pymnt']=all_d['issue_To_last_pymnt'].map(lambda x:x.days)
all_d['last_credit_pull_d_To_last_pymnt']=all_d['last_credit_pull_d_To_last_pymnt'].map(lambda x:x.days)
all_d['issue_To_last_credit_pull_d']=all_d['issue_To_last_credit_pull_d'].map(lambda x:x.days)


C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [161]:
add=['Unnamed: 0.1','issue_To_last_pymnt','last_credit_pull_d_To_last_pymnt','issue_To_last_credit_pull_d']
X_test_add=X_test_add[add]
X_train_add=X_train_add[add]

In [162]:
X_test_add_id=X_test_add['Unnamed: 0.1']
X_train_add_id=X_train_add['Unnamed: 0.1']

X_train_add=X_train_add.drop(columns=['Unnamed: 0.1'])
X_test_add=X_test_add.drop(columns=['Unnamed: 0.1'])

In [163]:
# 標準化
 # 标准化数据
ss = StandardScaler()
X_test_add_std= pd.DataFrame(ss.fit_transform(X_test_add),columns=X_test_add.columns,index=X_test_add.index)
 
    
ss = StandardScaler()
X_train_add_std= pd.DataFrame(ss.fit_transform(X_train_add),columns=X_train_add.columns,index=X_train_add.index)
    

In [164]:
X_test_add_std['Unnamed: 0.1']=X_test_add_id
X_train_add_std['Unnamed: 0.1']=X_train_add_id

In [165]:
X_train_add_std

,issue_To_last_pymnt,last_credit_pull_d_To_last_pymnt,issue_To_last_credit_pull_d,Unnamed: 0.1
2,0.755348,-0.678151,1.056917,1192524
9,-2.304065,-0.180010,-1.436296,2062903
13,-0.233663,-0.175333,-0.021179,653911
14,-1.641102,-0.610329,-0.639362,1985095
19,-0.483633,0.958932,-1.095551,1223705
...,...,...,...,...
2132787,0.918372,-0.395170,0.943335,1770366
2132789,-0.978138,0.530952,-1.093689,1195738
2132790,-1.228108,2.170374,-2.570252,178207
2132792,-1.065084,-0.610329,-0.244618,935


In [166]:
# 
X_train_more=pd.merge(X_train,X_train_add_std)
X_test_more=pd.merge(X_test,X_test_add_std)

In [167]:
# 補值

# 用 mean 補值
var=X_train_more.columns.tolist()

for i in range(0,len(var)): 
    X_train_more[str(var[i])]=X_train_more[str(var[i])].fillna(X_train_more[str(var[i])].mean())

In [168]:
# 用 mean 補值
var=X_test_more.columns.tolist()

for i in range(0,len(var)): 
    X_test_more[str(var[i])]=X_test_more[str(var[i])].fillna(X_test_more[str(var[i])].mean())


## 
X_train_more
X_test_more
y_train
y_test

# model

In [169]:
# mlp
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import itertools
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import Sequential
import keras
from keras.models import Sequential    #匯入線性堆疊模型之模組
from keras.layers import Dense
from keras import metrics#匯入緊密連接模組

In [170]:
X_train_more.to_csv(path+'\grade\B\X_train_more.csv')
X_test_more.to_csv(path+'\grade\B\X_test_more.csv')

In [171]:
X_train_more=X_train_more.drop(columns=['Unnamed: 0.1'])
X_test_more=X_test_more.drop(columns=['Unnamed: 0.1'])

In [191]:
y_train=pd.read_csv(path+'/grade/B/y_trainB.csv',index_col=0)
y_test=pd.read_csv(path+'/grade/B/y_testB.csv',index_col=0)

# 處理 y
le = LabelEncoder()
le.fit(y_train['loan_status'].unique())  
y_train['ans']=le.transform(y_train['loan_status'])


In [192]:
# 分 train0 test0
X_train0, X_test0, y_train0, y_test0 = train_test_split(X_train_more, y_train, test_size=0.2, random_state=0)

# M 1

In [174]:
from imblearn.over_sampling import ADASYN 
X4, y4 = ADASYN().fit_resample(X_train0, y_train0.ans)  

In [175]:
y_train0.ans.shape

(495496,)

In [176]:
y_train0.ans.sum()

49165

In [177]:
y4.shape

(887721,)

In [178]:
y4.sum()

441390

In [179]:
X4.shape

(887721, 82)

In [189]:
X4.to_csv(path+'\grade\B\X4.csv')
y4.to_csv(path+'\grade\B\y4.csv')

In [180]:
X_train0=X4
y_train0=y4
y_train0_0 = keras.utils.to_categorical(y_train0, 2)

In [181]:
from keras import backend as K
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.utils import np_utils

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [182]:
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.utils import np_utils

In [183]:
model03 = Sequential() #建立模型
model03.add(Dense(input_dim = X_train0.shape[1], output_dim = 48)) #添加输入层、隐藏层的连接
model03.add(Activation('tanh')) #以Relu函数为激活函数

model03.add(Dense(input_dim = 48, output_dim = 48)) #添加隐藏层、隐藏层的连接
model03.add(Activation('relu')) #以Relu函数为激活函数
model03.add(Dropout(0.2))

model03.add(Dense(input_dim = 48, output_dim = 36)) #添加隐藏层、隐藏层的连接
model03.add(Activation('relu')) #以Relu函数为激活函数
model03.add(Dropout(0.2))
model03.add(Dense(input_dim = 36, output_dim = 36)) #添加隐藏层、隐藏层的连接
model03.add(Activation('relu')) #以Relu函数为激活函数

model03.add(Dense(input_dim = 36, output_dim = 12)) #添加隐藏层、隐藏层的连接
model03.add(Activation('relu')) #以Relu函数为激活函数
model03.add(Dense(input_dim = 12, output_dim = 12)) #添加隐藏层、隐藏层的连接
model03.add(Activation('relu')) #以Relu函数为激活函数


model03.add(Dense(input_dim = 12, output_dim = 2)) #添加隐藏层、输出层的连接
model03.add(Activation('sigmoid')) #以sigmoid函数为激活函数

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=82, units=48)`
  
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=48, units=48)`
  """
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=48, units=36)`
  if __name__ == '__main__':
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=36, units=36)`
  if sys.path[0] == '':
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=36, units=12)`
  from ipykernel import kernelapp as app
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to t

In [184]:
model03.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
               metrics=[f1_m])

train_history=model03.fit(X_train0, y_train0_0, validation_split=0.3, epochs=70, batch_size=200, verbose=2)  

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 621404 samples, validate on 266317 samples
Epoch 1/70
 - 10s - loss: 0.1107 - f1_m: 0.9606 - val_loss: 0.0790 - val_f1_m: 0.9675
Epoch 2/70
 - 9s - loss: 0.0673 - f1_m: 0.9782 - val_loss: 0.0644 - val_f1_m: 0.9747
Epoch 3/70
 - 9s - loss: 0.0592 - f1_m: 0.9811 - val_loss: 0.0345 - val_f1_m: 0.9864
Epoch 4/70
 - 9s - loss: 0.0551 - f1_m: 0.9829 - val_loss: 0.0468 - val_f1_m: 0.9814
Epoch 5/70
 - 9s - loss: 0.0525 - f1_m: 0.9837 - val_loss: 0.0483 - val_f1_m: 0.9809
Epoch 6/70
 - 9s - loss: 0.0508 - f1_m: 0.9844 - val_loss: 0.0629 - val_f1_m: 0.9783
Epoch 7/70
 - 9s - loss: 0.0493 - f1_m: 0.9848 - val_loss: 0.0531 - val_f1_m: 0.9841
Epoch 8/70
 - 9s - loss: 0.0479 - f1_m: 0.9855 - val_loss: 0.0615 - val_f1_m: 0.9775
Epoch 9/70
 - 9s - loss: 0.0470 - f1_m: 0.9857 - val_loss: 0.0540 - val_f1_m: 0.9812
Epoch 10/70
 - 9s - loss: 0.0461 - f1_m: 0.9860 - val_loss: 0.0352 - val_f1_m: 0.9886
E

In [185]:
prediction=model03.predict_classes(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

0.9418351997325757
0.9887627750779018


In [186]:
X_test.shape

(155106, 80)

In [188]:
## predict Y
y_pred=model03.predict_classes(X_test_more)
y_ans = pd.DataFrame(y_pred,columns=['predict'],index=y_test.index)
y_output=pd.merge(y_test,y_ans,left_index=True,right_index=True)

y_output.to_csv(path+'\grade\B\y_output_MLP_B_02.csv')

# M 2

In [193]:
from imblearn.over_sampling import BorderlineSMOTE
oversample = BorderlineSMOTE()
X3, y3 = oversample.fit_resample(X_train0, y_train0.ans)

In [194]:
print(y_train0.ans.sum())
print(y_train0.ans.shape)
print(y3.sum())
print(y3.shape)

49165
(495496,)
446331
(892662,)


In [195]:
X_train0=X3
y_train0=y3
y_train0_0 = keras.utils.to_categorical(y_train0, 2)

In [196]:
from keras import backend as K
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.utils import np_utils

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [198]:
model04 = Sequential() #建立模型
model04.add(Dense(input_dim = X_train0.shape[1], output_dim = 48)) #添加输入层、隐藏层的连接
model04.add(Activation('tanh')) #以Relu函数为激活函数

model04.add(Dense(input_dim = 48, output_dim = 48)) #添加隐藏层、隐藏层的连接
model04.add(Activation('relu')) #以Relu函数为激活函数
model04.add(Dropout(0.2))

model04.add(Dense(input_dim = 48, output_dim = 36)) #添加隐藏层、隐藏层的连接
model04.add(Activation('relu')) #以Relu函数为激活函数
model04.add(Dropout(0.2))
model04.add(Dense(input_dim = 36, output_dim = 36)) #添加隐藏层、隐藏层的连接
model04.add(Activation('relu')) #以Relu函数为激活函数

model04.add(Dense(input_dim = 36, output_dim = 12)) #添加隐藏层、隐藏层的连接
model04.add(Activation('relu')) #以Relu函数为激活函数
model04.add(Dense(input_dim = 12, output_dim = 12)) #添加隐藏层、隐藏层的连接
model04.add(Activation('relu')) #以Relu函数为激活函数

model04.add(Dense(input_dim = 12, output_dim = 2)) #添加隐藏层、输出层的连接
model04.add(Activation('sigmoid')) #以sigmoid函数为激活函数

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=82, units=48)`
  
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=48, units=48)`
  """
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=48, units=36)`
  if __name__ == '__main__':
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=36, units=36)`
  if sys.path[0] == '':
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=36, units=12)`
  from ipykernel import kernelapp as app
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to t

In [199]:
model04.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
               metrics=[f1_m])

train_history=model04.fit(X_train0, y_train0_0, validation_split=0.3, epochs=70, batch_size=200, verbose=2)  

Train on 624863 samples, validate on 267799 samples
Epoch 1/70
 - 10s - loss: 0.0914 - f1_m: 0.9688 - val_loss: 0.0510 - val_f1_m: 0.9774
Epoch 2/70
 - 10s - loss: 0.0576 - f1_m: 0.9821 - val_loss: 0.0444 - val_f1_m: 0.9804
Epoch 3/70
 - 10s - loss: 0.0517 - f1_m: 0.9843 - val_loss: 0.0448 - val_f1_m: 0.9801
Epoch 4/70
 - 9s - loss: 0.0486 - f1_m: 0.9855 - val_loss: 0.0282 - val_f1_m: 0.9889
Epoch 5/70
 - 9s - loss: 0.0464 - f1_m: 0.9863 - val_loss: 0.0263 - val_f1_m: 0.9896
Epoch 6/70
 - 9s - loss: 0.0450 - f1_m: 0.9868 - val_loss: 0.0313 - val_f1_m: 0.9864
Epoch 7/70
 - 10s - loss: 0.0439 - f1_m: 0.9872 - val_loss: 0.0274 - val_f1_m: 0.9900
Epoch 8/70
 - 10s - loss: 0.0430 - f1_m: 0.9875 - val_loss: 0.0146 - val_f1_m: 0.9934
Epoch 9/70
 - 9s - loss: 0.0424 - f1_m: 0.9878 - val_loss: 0.0274 - val_f1_m: 0.9892
Epoch 10/70
 - 9s - loss: 0.0421 - f1_m: 0.9880 - val_loss: 0.0152 - val_f1_m: 0.9934
Epoch 11/70
 - 9s - loss: 0.0418 - f1_m: 0.9879 - val_loss: 0.0189 - val_f1_m: 0.9924
Epoch 

In [200]:
prediction=model04.predict_classes(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

0.9442394767734361
0.9892632836592021


In [201]:
y_pred=model04.predict_classes(X_test_more)
y_ans = pd.DataFrame(y_pred,columns=['predict'],index=y_test.index)
y_output=pd.merge(y_test,y_ans,left_index=True,right_index=True)

y_output.to_csv(path+'\grade\B\y_output_MLP_B_03.csv')

# M 3

In [ ]:
# 分 train0 test0
X_train0, X_test0, y_train0, y_test0 = train_test_split(X_train_more, y_train, test_size=0.2, random_state=0)

In [ ]:
from imblearn.over_sampling import SVMSMOTE
oversample = SVMSMOTE()
X1, y1 = oversample.fit_resample(X_train0, y_train0.ans)

In [ ]:
print(y_train0.ans.sum())
print(y_train0.ans.shape)
print(y1.sum())
print(y1.shape)

In [ ]:
X1.to_csv(path+'\grade\B\X1.csv')
y1.to_csv(path+'\grade\B\y1.csv')

In [ ]:
X_train0=X1
y_train0=y1
y_train0_0 = keras.utils.to_categorical(y_train0, 2)

In [ ]:
from keras import backend as K
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.utils import np_utils

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
model02 = Sequential() #建立模型
model02.add(Dense(input_dim = X_train0.shape[1], output_dim = 48)) #添加输入层、隐藏层的连接
model02.add(Activation('tanh')) #以Relu函数为激活函数

model02.add(Dense(input_dim = 48, output_dim = 48)) #添加隐藏层、隐藏层的连接
model02.add(Activation('relu')) #以Relu函数为激活函数
model02.add(Dropout(0.2))

model02.add(Dense(input_dim = 48, output_dim = 36)) #添加隐藏层、隐藏层的连接
model02.add(Activation('relu')) #以Relu函数为激活函数
model02.add(Dropout(0.2))
model02.add(Dense(input_dim = 36, output_dim = 36)) #添加隐藏层、隐藏层的连接
model02.add(Activation('relu')) #以Relu函数为激活函数

model02.add(Dense(input_dim = 36, output_dim = 12)) #添加隐藏层、隐藏层的连接
model02.add(Activation('relu')) #以Relu函数为激活函数
model02.add(Dense(input_dim = 12, output_dim = 12)) #添加隐藏层、隐藏层的连接
model02.add(Activation('relu')) #以Relu函数为激活函数


model02.add(Dense(input_dim = 12, output_dim = 2)) #添加隐藏层、输出层的连接
model02.add(Activation('sigmoid')) #以sigmoid函数为激活函数

In [ ]:
model02.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
               metrics=[f1_m])

train_history=model02.fit(X_train0, y_train0_0, validation_split=0.3, epochs=70, batch_size=200, verbose=2)  

In [ ]:
prediction=model02.predict_classes(X_test0) 

y_true=y_test0.ans
print(f1_score(y_true, prediction, average='binary'))
print(accuracy_score(y_true, prediction))

In [ ]:
y_pred=model02.predict_classes(X_test_more)
y_ans = pd.DataFrame(y_pred,columns=['predict'],index=y_test.index)
y_output=pd.merge(y_test,y_ans,left_index=True,right_index=True)

y_output.to_csv(path+'\grade\B\y_output_MLP_B_04.csv')